In [1]:
# humbug main imports

import os
import pandas as pd
import sys
sys.path.insert(0, os.path.abspath('../lib'))
import config
from evaluate import get_results
import numpy as np

# Troubleshooting and visualisation
import IPython.display as ipd


In [2]:
# humbug lib imports
from sklearn.metrics import accuracy_score
from PyTorch import config_pytorch
from datetime import datetime
import math
import pickle

from torch.utils.data import TensorDataset, DataLoader
import torch.nn.functional as F
import torch.nn as nn
import torch
import torch.optim as optim
import numpy as np
from sklearn.metrics import accuracy_score
from datetime import datetime
import os

import matplotlib
import matplotlib.pyplot as plt
import sklearn
from sklearn.metrics import confusion_matrix, classification_report
from sklearn.metrics import precision_recall_curve, plot_precision_recall_curve
from sklearn.metrics import average_precision_score
import sys

from tqdm.notebook import tqdm

In [3]:
# additional pytorch tools
import random
import torchaudio
import torchaudio.transforms as T
import torchvision.transforms as VT
from torch.cuda.amp import autocast, GradScaler
from timm.scheduler.cosine_lr import CosineLRScheduler
import timm
import timm.optim
from timm.loss import BinaryCrossEntropy
from timm.utils import NativeScaler
from timm.models import model_parameters
from glob import glob

In [20]:
!pip install torch-audiomentations

Looking in indexes: https://pypi.org/simple, https://pypi.ngc.nvidia.com
     |████████████████████████████████| 47 kB 7.3 MB/s eta 0:00:011
     |████████████████████████████████| 58 kB 56.6 MB/s eta 0:00:01
  Created wheel for julius: filename=julius-0.2.6-py3-none-any.whl size=20966 sha256=65622210d18a92a92f4d55fc3463ee5478444192209bd483fa32fa17d75768aa
  Stored in directory: /tmp/pip-ephem-wheel-cache-ku5ubl08/wheels/0c/3e/a6/7bd33f66da9efe0a0288706a2ab190a0879190b7b159644ce2
Successfully built julius


In [4]:
#convnext_small
timm.list_models('convnext_small*' , pretrained= True)


['convnext_small',
 'convnext_small_384_in22ft1k',
 'convnext_small_in22ft1k',
 'convnext_small_in22k']

In [16]:
def pad_mean(x_temp,rate = config.rate, min_length = config.min_duration ):
    print("inside padding mean...")
    x_mean = torch.mean(x_temp)
    print("X_mean = " + str(x_mean))
    left_pad_amt = int((rate*min_length-x_temp.shape[1])//2)
    print("left_pad_amt = " + str(left_pad_amt))
    left_pad = torch.zeros(1,left_pad_amt) #+ (0.1**0.5)*torch.randn(1, left_pad_amt)
    print("left_pad shape = " + str(left_pad.shape))
    left_pad_mean_add = left_pad + x_mean
    print("left_pad_mean shape = " + str(left_pad_mean_add))
    print("sum of left pad mean add = " + str(torch.sum(left_pad_mean_add)))
    
    right_pad_amt = int(rate*min_length-x_temp.shape[1]-left_pad_amt)
    right_pad = torch.zeros(1,right_pad_amt)# + (0.1**0.5)*torch.randn(1, right_pad_amt)
    print("right_pad shape = " + str(right_pad.shape))
    right_pad_mean_add = right_pad + x_mean
    print("right_pad_mean shape = " + str(right_pad_mean_add))
    print("sum of right pad mean add = "  + str(torch.sum(right_pad_mean_add)))
    
    
    
    f = torch.cat([left_pad,x_temp,right_pad],dim=1)[0]
    f = f.unsqueeze(dim = 0)
    #print("returning a tensor of shape = " + str(f.shape))
    return(f)

In [19]:
# create a .1 sec signal
signal_durn = int(.1*8000)
sig = torch.rand(1,signal_durn)
print(sig.shape)
sig_mean = torch.mean(sig)
print(sig_mean)
print(torch.sum(sig))

f = pad_mean(sig)
print(f.shape)


torch.Size([1, 800])
tensor(0.5076)
tensor(406.0925)
inside padding mean...
X_mean = tensor(0.5076)
left_pad_amt = 7280
left_pad shape = torch.Size([1, 7280])
left_pad_mean shape = tensor([[0.5076, 0.5076, 0.5076,  ..., 0.5076, 0.5076, 0.5076]])
sum of left pad mean add = tensor(3695.4417)
right_pad shape = torch.Size([1, 7280])
right_pad_mean shape = tensor([[0.5076, 0.5076, 0.5076,  ..., 0.5076, 0.5076, 0.5076]])
sum of right pad mean add = tensor(3695.4417)
torch.Size([1, 15360])


In [43]:
!pwd

/dli/task/Capstone-HumBug/notebooks


In [62]:
from torch_audiomentations import Compose,AddBackgroundNoise , AddColoredNoise , ApplyImpulseResponse,PeakNormalization,TimeInversion

In [63]:
import torch
from torch_audiomentations import Compose, Gain, PolarityInversion,AddColoredNoise,ApplyImpulseResponse,PeakNormalization
# Initialize augmentation callable
apply_augmentation = Compose(
    transforms=[PolarityInversion(p=0.5 ,output_type = 'tensor'),AddColoredNoise(),AddBackgroundNoise( background_paths = "../data/audio/noise") , PeakNormalization(apply_to="only_too_loud_sounds"),TimeInversion(output_type = 'tensor')
    ]
)


In [55]:
errors = []
for i, val in enumerate(y_test_B.squeeze()):
    if val != np.argmax(y_preds_all[0],-1)[i]:
        errors.append(i)

torch.Size([1, 1, 15360])

In [56]:
perturbed_audio_samples = apply_augmentation(f_unsq, sample_rate=8000)

In [57]:
perturbed_audio_samples.shape

torch.Size([1, 1, 15360])

In [65]:
5%2

1

In [5]:
timm.models.convnext_small

<function timm.models.convnext.convnext_small(pretrained=False, **kwargs)>

In [6]:
model = timm.create_model('convnext_small',pretrained= True)

In [7]:
model

ConvNeXt(
  (stem): Sequential(
    (0): Conv2d(3, 96, kernel_size=(4, 4), stride=(4, 4))
    (1): LayerNorm2d((96,), eps=1e-06, elementwise_affine=True)
  )
  (stages): Sequential(
    (0): ConvNeXtStage(
      (downsample): Identity()
      (blocks): Sequential(
        (0): ConvNeXtBlock(
          (conv_dw): Conv2d(96, 96, kernel_size=(7, 7), stride=(1, 1), padding=(3, 3), groups=96)
          (norm): LayerNorm((96,), eps=1e-06, elementwise_affine=True)
          (mlp): Mlp(
            (fc1): Linear(in_features=96, out_features=384, bias=True)
            (act): GELU()
            (drop1): Dropout(p=0.0, inplace=False)
            (fc2): Linear(in_features=384, out_features=96, bias=True)
            (drop2): Dropout(p=0.0, inplace=False)
          )
          (drop_path): Identity()
        )
        (1): ConvNeXtBlock(
          (conv_dw): Conv2d(96, 96, kernel_size=(7, 7), stride=(1, 1), padding=(3, 3), groups=96)
          (norm): LayerNorm((96,), eps=1e-06, elementwise_affine

In [8]:
model.default_cfg

{'url': 'https://dl.fbaipublicfiles.com/convnext/convnext_small_1k_224_ema.pth',
 'num_classes': 1000,
 'input_size': (3, 224, 224),
 'pool_size': (7, 7),
 'crop_pct': 0.875,
 'interpolation': 'bicubic',
 'mean': (0.485, 0.456, 0.406),
 'std': (0.229, 0.224, 0.225),
 'first_conv': 'stem.0',
 'classifier': 'head.fc',
 'architecture': 'convnext_small'}

In [9]:
m = timm.create_model('convnext_small',pretrained= True , in_chans = 1)
x = torch.randn(1,1,224,224)
m(x).shape

torch.Size([1, 1000])

In [10]:
model.head.fc

Linear(in_features=768, out_features=1000, bias=True)

In [11]:
model.get_classifier()

Linear(in_features=768, out_features=1000, bias=True)

In [12]:
def train(lr: float,mining: str, batch_size: int, epochs: int,  input_size: int, shuffle: bool, save_dir: str,):
    
    model = Model(lr=lr,mining=mining, )
    train_dataloader, val_dataloader = get_dataloaders( batch_size=batch_size, input_size=input_size, shuffle=shuffle )
    early_stopping = EarlyStopping(monitor="validation_loss",patience=50,  )
    
    trainer = pl.Trainer(
        gpus=1 if torch.cuda.is_available() else 0,
        max_epochs=epochs,
        callbacks=[early_stopping, ModelSummary(max_depth=3)],
        enable_checkpointing=False,
        log_every_n_steps=1,
    )

    Quaterion.fit(
        trainable_model=model,
        trainer=trainer,
        train_dataloader=train_dataloader,
        val_dataloader=val_dataloader,
    )

    model.save_servable(save_dir)

In [17]:
x = torch.tensor(8364)
print(x)
x1 = x.numpy()
print(x1)

tensor(8364)
8364


In [20]:
t = [1,4000,2,6,73, 554]
t.index(max(t))

1